In [252]:
# pip install plotly==4.3.0

import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
    
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.stattools import acf


#check the value of D
from statsmodels.tsa.stattools import adfuller
from numpy import log
import numpy as np
import datetime

In [253]:
str_csv = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv'
df = pd.read_csv(str_csv)


latitude = df['Lat'].tolist()
longitude = df['Long'].tolist()
cities = df['Province/State'].tolist()
lat_long = [[lat,long] for lat,long in zip(latitude,longitude)]
lat_long.pop()
dates = df.columns[5:]
df.loc['Current',:]= df[dates].sum(axis=0)


Current = df.loc['Current',dates].tolist()
last_date = dates[-1]
# print(last_date)

# print(cities,latitude,longitude)


In [254]:

df = pd.DataFrame(list(zip(dates,Current)),columns =['Date', 'Current'])

# print(df)
# print(df.head())
moving_avg = df.rolling(3).mean()
# print(len(dates),len(moving_avg))
moving_avg.insert(0,'Date',dates,True)
# print(moving_avg.head())


train = df.Current.tail(4)
last_fifteen = len(train)
# print(train)
test = df.Current.tail(15)


moving_avg = df.iloc[:,1].rolling(window=3).mean()
moving_avg.dropna(inplace=True)
model = ARIMA(train, order=(1, 1,0 ))
fitted = model.fit(disp=-1)
# print(moving_avg)


fc, se, conf = fitted.forecast(15, alpha=0.05)  # 95% conf


train = df.Current
last_fifteen = len(train)
# print(train)
test = df.Current.tail(15)


moving_avg = df.iloc[:,1].rolling(window=3).mean()
moving_avg.dropna(inplace=True)
model = ARIMA(train, order=(1, 1,0 ))
fitted = model.fit(disp=-1)
# print(moving_avg)


fc, sec, conf = fitted.forecast(15, alpha=0.05)  # 95% conf


In [255]:
dates2 = pd.date_range(last_date, periods=15)
dates2 = dates2.strftime("%m/%d/%y")
# dates2[0]

#### Bar chart of bank nifty close

In [256]:

fc_series = pd.Series(fc, index=test.index)
lower_series = pd.Series(conf[:, 0], index=test.index)
upper_series = pd.Series(conf[:, 1], index=test.index)

upper_list = upper_series.tolist()
decreasing = []
for i in range(len(upper_list)-1):
    diff = upper_list[i] - upper_list[i+1]
    decreasing.append(diff)
difference = []
for i in range(len(decreasing)-1):
    diff = decreasing[i] - decreasing[i+1]
    difference.append(diff)

add = 0
for i in range(len(difference)):
    add = add + (i**2+i**1.5)*abs(difference[i])
    difference[i] = add
difference.insert(0,0)    
difference.insert(0,0) 
 
lower_list = lower_series.tolist()
i = len(lower_list)

for i in range(len(lower_list)):
    lower_list[i] = lower_list[i] - difference[i]
    
lower_list = lower_list[1:]
    

fc_series = pd.DataFrame(list(zip(dates2[:len(fc_series)],fc_series)),columns =['Date','Current'])
lower_series = pd.DataFrame(list(zip(dates2[:len(fc_series)],lower_list)),columns =['Date','Current'])
upper_series = pd.DataFrame(list(zip(dates2[:len(fc_series)],upper_series)),columns =['Date','Current'])
moving_avg_series = pd.DataFrame(list(zip(dates[len(dates)-len(moving_avg):],moving_avg)),columns =['Date','Current'])


#### Plot The data on the graph

In [258]:

df_last_elem = pd.DataFrame({'Date': df.Date.tail(1), 'Current': df.Current.tail(1)})
lower_series = df_last_elem.append(lower_series)
upper_series = df_last_elem.append(upper_series)
fc_series = df_last_elem.append(fc_series)
# print(fc_series)

# print(result)
pred_y = df.Current.tail(1).tolist()
# print(pred_y)
train = df.Current.tail(4)
x = [x for x in range(len(train))]
y = train
z = np.polyfit(x, y, 1)
f = np.poly1d(z)
# print(f(28))
pred_x = [x for x in range(len(train),len(train)+15) ] 
for x in range(len(train),len(train)+14):
    pred_y.append(f(x)) 
new_fc_series = pd.DataFrame(list(zip(dates2[:len(fc_series)],pred_y)),columns =['Date','Current'])
new_fc_series = df_last_elem.append(new_fc_series)
# print(new__fc_series)

fig = go.Figure()
fig.add_trace(go.Scatter(
                x=df.Date,
                y=df['Current'],
                name="Total No of cases",
                line_color='deepskyblue',
                opacity=0.8))

# fig.add_trace(go.Scatter(
#                 x=moving_avg_series.Date,
#                 y=moving_avg_series['Current'],
#                 name="Moving Avg",
#                 line_color='yellow',
#                 opacity=0.8))

fig.add_trace(go.Scatter(
                x=new_fc_series.Date,
                y=new_fc_series.Current,
                name="Current rate",
                line_color='gray',
                opacity=0.8))

fig.add_trace(go.Scatter(
                x=lower_series.Date,
                y=lower_series['Current'],
                name="decreasing rate",
                line_color='green',
                opacity=0.8))

fig.add_trace(go.Scatter(
                x=upper_series.Date,
                y=upper_series['Current'],
                name="Increasing rate",
                line_color='red',
                opacity=0.8))


# Use date string to set xaxis range
fig.update_layout(
                  title_text ="Manually Set Date Range")
fig.show()


In [259]:
import keplergl
# w1 = keplergl.KeplerGl(height=500, data={"data_1": df}, config=config)
df_map = pd.DataFrame(
    {'City': cities,
     'Latitude': latitude,
     'Longitude': longitude})

w1 = keplergl.KeplerGl()


User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


In [260]:
config = {
    'version': 'v1',
    'config': {
        'mapState': {
            'latitude': 37.76209132041332,
            'longitude': -122.42590232651203,
            'zoom': 12.32053899007826
        }
    }
}
w1.add_data(data=df_map, name='corona points')
w1.config = config
w1.config

{'version': 'v1',
 'config': {'mapState': {'latitude': 37.76209132041332,
   'longitude': -122.42590232651203,
   'zoom': 12.32053899007826}}}

In [261]:
w1.save_to_html(data={'data_1': df_map}, config=config, file_name='first_map.html')

Map saved to first_map.html!
